In [20]:
# this is copied from tile.c and made a Python list
from minihack.tiles import GlyphMapper
from nle_toolbox.wrappers.replay import Replay
import pickle as pkl
import gym
from nle.nethack import ACTIONS

import sys
sys.path.append('../')
from envs.wrappers import MinihackWrapper



gm = GlyphMapper()


# state_dict = pkl.load(open('../test_present/0050kxwl.pkl', 'rb'))
# env = Replay(gym.make('NetHackChallenge-v0'))

state_dict = pkl.load(open('../replays/_ox7refk.pkl', 'rb'))
env = Replay(MinihackWrapper(gym.make('MiniHack-CorridorBattle-v0', actions=ACTIONS)))

# force the seed
env.seed(seed=state_dict['seed'])
obs = env.reset()

In [21]:
import IPython.display
import PIL.Image
import numpy as np
import cv2
from tqdm import tqdm


TILE_SIZE = (16, 16, 3)
DUNGEON_SIZE = (28, 80)


font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
fontColor = (255,255,255)
thickness = 1
lineType = 2


def render_text(array, text, bottom_left_corner):
    cv2.putText(array, text, 
            bottom_left_corner, 
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType
        )


def render(obs):
    # two line offset + one line text
    img = [np.zeros((3 * TILE_SIZE[0], 80 * TILE_SIZE[1], 3), dtype=np.uint8)]

    # 21 x 80
    for row in obs['glyphs']:
        img_row = []
        for glyph in row:
            img_row.append(gm.glyph_id_to_rgb(glyph))
        img_row.append(np.zeros(TILE_SIZE, dtype=np.uint8))
        img_row = np.concatenate(img_row, axis=1)
        img.append(img_row)
    
    # one line offset + two lines text + bottom line
    img.append(np.zeros((4 * TILE_SIZE[0], 80 * TILE_SIZE[1], 3), dtype=np.uint8))
    img = np.concatenate(img, axis=0)
    
    for line_id, line in enumerate(obs['tty_chars'][22:]):
        line = bytes(line).decode('ascii')
        bottom_left_corner = (16, 16 * (25 + line_id * 1))
        render_text(img, line, bottom_left_corner)
    
    line = bytes(obs['tty_chars'][0]).decode('ascii')
    bottom_left_corner = (16, 16 * 3)
    render_text(img, line, bottom_left_corner)

    return img


fps = 10 #25
out = cv2.VideoWriter(
    'output.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    fps, 
    (TILE_SIZE[1] * DUNGEON_SIZE[1], TILE_SIZE[0] * DUNGEON_SIZE[0])
)

print(state_dict['actions'])
for i, act in tqdm(enumerate(state_dict['actions']), total=len(state_dict['actions'])):
    obs, reward, done, info = env.step(act)
    img = render(obs)
    out.write(img)

out.release()


[25, 4, 4, 5, 108, 102, 75, 40, 1, 40, 1, 40, 1, 40, 1, 1, 1, 1, 40, 1, 40, 1, 40, 1, 40, 1, 40, 1, 1, 40, 1, 40, 1, 40, 1, 40, 1, 40, 1, 1, 40, 1, 40, 1, 40, 1, 108, 102, 75, 108, 102, 75, 108, 102, 75, 108, 102, 75, 40, 1, 40, 1, 40, 1, 1, 1, 1, 4, 5, 1, 1, 4, 5, 4, 5, 4, 5, 5, 4, 4, 1, 1, 1, 1, 5, 5, 0, 1]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:00<00:00, 216.29it/s]
